# Data Mining Project

## Knowledge Extraction Pipeline

This notebooks defines a series os steps that end up in the production of the desired prediction, including Data Preparation, Modeling and Evaluation, according to CRISP-DM guidelines.
For information regarding Data Understanding, please refer to [Data Understanding](data_understanding.ipynb).

#### Dependencies

The code block below defines the major dependencies for the project.
To make sure you are set up, please run the following command in order to update dependencies:

```bash
pip install -r requirements.txt
```

We chose to use a set of technologies that we were familiar with and should be adequate for the problem at hand.
These include **sklearn** to model the data, **matplotlib** and **seaborn** to create graphics and **pandas** to better read the data.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os

from utils.files import *


## Data Understanding


### Related Work

Sports related predictions are a fairly common problem.
It serves of value for different entities, such as bookmakers, sports teams and fans.
This fact together with the recent increase in the availability of data justifies the employment of machine learning techniques to the problem. [<a href="#ref1">1</a>]

The problem of predicting the outcome of a basketball game has been approached in different ways.
A common take on the subject is to try and predict the outcome of a single game, as opposed to the set of qualified teams.
Nevertheless, some similarities found between solutions were the use of machine learning algorithms and of similar attributes (rebounds, free throws, turn overs, etc).

In [<a href="#ref2">2</a>] the authors identify the characteristic high-dimensionality of the problem, and employ a Support Vector Machine Algorithm that predicted the outcome of a game with 88% accuracy.

Among common attributes, the author of [<a href="#ref3">3</a>] found that the most important ones were Free Throws, Offensive Rebounds, Turn Overs and +/- (Plus Minus).
They were also able to predict the champion team with an 86% recall using Random Forest.


Finally, the authors of [<a href="#ref4">4</a>] used a Naive Bayes Classifier to predict the outcome of games with 67% accuracy.

## Data Preparation

TODO: add text about data set (summary from data exploration)

TODO: add text about which transformations were made

In [2]:
df = pd.read_csv(os.path.join(DATA_PATH, DATA_MERGED))
df.head()

,year,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,o_reb,o_asts,...,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,dq
0,10,1,895.0,2258.0,542.0,725.0,202.0,598.0,1077.0,492.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
1,2,1,812.0,1903.0,431.0,577.0,131.0,386.0,935.0,551.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
2,3,1,746.0,1780.0,410.0,528.0,153.0,428.0,948.0,467.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
3,4,1,770.0,1790.0,490.0,663.0,211.0,527.0,955.0,496.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
4,5,0,787.0,1881.0,456.0,590.0,187.0,517.0,971.0,499.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054


## Modeling and Evaluation

The following block imports general utility functions (defined in `utils/modeling.py`) that will serve to model the data and access results.

We chose to create the training and testing subsets in a temporal fashion.
The reason being it wouldn't make sense to scatter data from different years, since our game data is chronological.
As an example, we can train the model with the first 9 years and use the 10th and last to test the model's predictions.

In [3]:
from utils.modeling import *

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm

models = {
    "Decision Tree" : DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=20),
    "Random Forest" : RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42),
    "Naive Bayes" : GaussianNB(),
    "Neural Net" : MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42),
    "SVM" : svm.SVC(kernel='linear', C=1, random_state=42)
    }

# iterate all models and store results in a df
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1'])
for name, model in models.items():
    res = runModel(df, model, test_year=10)
    results.loc[len(results)] = [name] + res.toRow()

results
    

,Model,Accuracy,Precision,Recall,F1
0,Decision Tree,0.6875,0.666667,0.888889,0.761905
1,Random Forest,0.4375,0.500000,0.777778,0.608696
2,Naive Bayes,0.5000,0.555556,0.555556,0.555556
3,Neural Net,0.5625,0.562500,1.000000,0.720000
4,SVM,0.6875,0.700000,0.777778,0.736842


### References

<a id="ref1"></a> [1] Bunker, R. P., & Thabtah, F. (2019). A machine learning framework for sport result prediction. Applied Computing and Informatics, 15(1), 27-33. https://doi.org/10.1016/j.aci.2017.09.005

<a id="ref2"></a> [2] Jadhav, A. (2016). Predicting the NBA playoff using SVM. CORE. https://core.ac.uk/display/230494997?utm_source=pdf&utm_medium=banner&utm_campaign=pdf-decoration-v1

<a id="ref3"></a> [3] Jien, O. W. (2022, January 5). Prediction model for NBA championship by Machine Learning. Medium. https://medium.com/@weinjien99/prediction-model-for-nba-championship-by-machine-learning-8e8884ea72c8

<a id="ref4"></a> [4] D. Miljković, L. Gajić, A. Kovačević and Z. Konjović, "The use of data mining for basketball matches outcomes prediction," IEEE 8th International Symposium on Intelligent Systems and Informatics, Subotica, Serbia, 2010, pp. 309-312, doi: 10.1109/SISY.2010.5647440.
